In [1]:
import pandas as pd
import yaml
import tomlkit  
import re

In [2]:
input_excel = "/workspaces/k8sconfig_edit/input_tom.xlsx"
# input_excel = "/workspaces/k8sconfig_edit/tom_edit/config.toml.xlsx" 

In [3]:
def convert_excel_to_yaml(excel_file):
    df = pd.read_excel(excel_file)

    yaml_dict = {}
    
    for _, row in df.iterrows():
        keys = row["Parameter"].split(".")
        value = row["Setup Value"]

        flag = False

        # Chuyển đổi giá trị thành kiểu phù hợp  
        if pd.isna(value):  # Kiểm tra nếu value là NaN (trống)  
            value = ""  # Gán None vào value thay vì bỏ qua  
        elif isinstance(value, str) and value.upper() == "TRUE":  
            value = True  
        elif isinstance(value, str) and value.upper() == "FALSE":  
            value = False  
        elif isinstance(value, str) and value == '[]':
            value = []
        elif isinstance(value, float) and value.is_integer():  
            value = int(value)  
        

        
        temp = yaml_dict
        
        
        for key in keys[:-1]:
            if key == "io" and keys[keys.index(key)-1] == "timeouts" :
                
                index = keys.index(key)  
                spec = '.'.join(keys[index:])
                # spec = \" + spec + \"
                # last_key = row["Parameter"]  
                temp[spec] = value
                flag = True
                break 

            if "[" in key and "]" in key: 
                base_key, index = key.split("[")
                index = int(index.rstrip("]"))
                temp = temp.setdefault(base_key, [])
                while len(temp) <= index:
                    temp.append({})
                temp = temp[index]
            else:
                temp = temp.setdefault(key, {})

        if flag:
            continue

        last_key = keys[-1]
        if "[" in last_key and "]" in last_key: 
            base_key, index = last_key.split("[")
            index = int(index.rstrip("]"))
            temp = temp.setdefault(base_key, [])
            while len(temp) <= index:
                temp.append(None)
            temp[index] = value
        else:
            temp[last_key] = value


    # Thay vì ghi vào file, ta trả về chuỗi YAML
    return yaml.dump(yaml_dict, default_flow_style=False, sort_keys=False)

if __name__ == "__main__":

    # Chuyển đổi thành định dạng YAML (không ghi vào file)
    yaml_content = convert_excel_to_yaml(input_excel)


In [4]:
# Hàm chuyển đổi file YAML sang TOML  
def yaml_to_toml(yaml_content):  
    # Chuyển đổi chuỗi YAML thành dữ liệu Python
    yaml_data = yaml.safe_load(yaml_content)

    # Tạo một tài liệu TOML mới  
    toml_data = tomlkit.document()  

    # Đưa dữ liệu từ YAML vào TOML đồng thời giữ nguyên thứ tự  
    for key, value in yaml_data.items():  
        toml_data[key] = value  

    # Tạo chuỗi TOML từ tài liệu TOML
    toml_output = tomlkit.dumps(toml_data)

    # Thêm lùi đầu dòng cho mỗi dòng giá trị trong bản thân chuỗi đầu ra  
    # Mỗi dòng sẽ là một phần tử trong danh sách lines
    lines = toml_output.splitlines()  
    for i in range(len(lines)):  

        flag = False

        #Kiểm tra xem dòng có dạng '[  ]'
        if lines[i].startswith("[[") and "]]" in lines[i]:
            flag = True
            #Tìm vị trí dấu chấm đầu tiên
            start_idx = lines[i].find(".")
            # Nếu có một dấu chấm và nó không phải là ký tự cuối cùng
            if start_idx != -1 and start_idx < len(lines[i]) - 1:
                # Thêm dấu ngoặc kép sau dấu chấm đầu tiên
                first_part = lines[i][:start_idx + 1]
                rest_part = lines[i][start_idx + 1:]
                
                # Tìm vị trí của '.cri' hoặc '.v1.cri'
                cri_idx2 = rest_part.find('.local')
                
                if cri_idx2 != -1 and cri_idx2 + 6 < len(rest_part):
                    # Chèn ngoặc kép sau 'cri'
                    before_cri = rest_part[:cri_idx2 + 6]  # Bao gồm '.cri'
                    after_cri = rest_part[cri_idx2 + 6:]   # Phần còn lại sau '.cri'
                    lines[i] = first_part + '"' + before_cri + '"' + after_cri

        if flag:
            if '=' in lines[i]:  # chỉ lùi những dòng có hình dạng key=value  
                lines[i] = '  ' + lines[i]  
            continue


        #Kiểm tra xem dòng có dạng '[  ]'
        if lines[i].startswith("[") and "]" in lines[i]:
            #Tìm vị trí dấu chấm đầu tiên
            start_idx = lines[i].find(".")
            # Nếu có một dấu chấm và nó không phải là ký tự cuối cùng
            if start_idx != -1 and start_idx < len(lines[i]) - 1:
                # Thêm dấu ngoặc kép sau dấu chấm đầu tiên
                first_part = lines[i][:start_idx + 1]
                rest_part = lines[i][start_idx + 1:]
                
                # Tìm vị trí của '.cri' hoặc '.v1.cri'
                cri_idx = rest_part.find('.cri')
                cri_idx1 = rest_part.find(']')
                
                if cri_idx != -1 and cri_idx + 4 < len(rest_part):
                    # Chèn ngoặc kép sau 'cri'
                    before_cri = rest_part[:cri_idx + 4]  # Bao gồm '.cri'
                    after_cri = rest_part[cri_idx + 4:]   # Phần còn lại sau '.cri'
                    lines[i] = first_part + '"' + before_cri + '"' + after_cri
                else:
                    # Chèn ngoặc kép sau 'cri'
                    before_cri1 = rest_part[:cri_idx1]  
                    lines[i] = first_part + '"' + before_cri1 + '"' + ']'

                    # # Nếu không tìm thấy '.cri', chỉ thêm dấu ngoặc kép sau dấu chấm đầu tiên
                    # lines[i] = first_part + '"' + rest_part + '"'

        if '=' in lines[i]:  # chỉ lùi những dòng có hình dạng key=value  
            lines[i] = '  ' + lines[i]  

    # Trả về chuỗi TOML đã chỉnh sửa
    return "\n".join(lines)
    
# Thực hiện chuyển đổi và lấy chuỗi TOML
toml_content = yaml_to_toml(yaml_content)

In [5]:
def transform_toml_content(toml_content):
    lines = toml_content.splitlines()
    transformed_lines = []
    plugins_section_added = False
    stream_processors_section_added = False
    
    for line in lines:
        
        # Thêm phần [plugins] nếu chưa có và gặp dòng [plugins. đầu tiên
        if line.startswith('[plugins.') and not plugins_section_added:
            transformed_lines.append('[plugins]\n')
            plugins_section_added = True
        elif line.startswith('[stream_processors.') and not stream_processors_section_added:
            transformed_lines.append('[stream_processors]\n')
            stream_processors_section_added = True

        
        
        # Thêm dòng vào danh sách kết quả
        transformed_lines.append(line)
    
    # Kết hợp các dòng trở lại thành một chuỗi
    return '\n'.join(transformed_lines)


 # Chuyển đổi nội dung TOML
transformed_content = transform_toml_content(toml_content)   

In [6]:


def split_str(input_str):
    # Bước 1: Loại bỏ dấu [ và ]  
    cleaned_str = input_str.strip("[]")  

    # Bước 2: Tách chuỗi bởi dấu "  
    parts = cleaned_str.split('"')  

    # Bước 3: Xử lý và tạo danh sách kết quả  
    result = []  

    for i, part in enumerate(parts):  
        if i % 2 == 0:  # Phần bên ngoài dấu "  
            sub_parts = part.split('.')  
            result.extend([sub_part for sub_part in sub_parts if sub_part])  # Thêm các phần không rỗng  
        else:  # Phần nằm trong dấu "  
            result.append(part)  # Thêm phần nằm giữa dấu "  

    # Bước 4: Kết hợp các phần bên trong  
    if len(result) > 1:  
        # Kết hợp tất cả các phần nằm trong dấu "  
        combined_inner = '.'.join(result[1:2])  # Chỉ cần phần nằm trong dấu "  
        result = [result[0], combined_inner] + result[2:]  # Thay thế và giữ lại các phần còn lại  

    # In kết quả  
    return result

def same(all_values, i):
    # Kiểm tra số lượng giá trị giống nhau giữa các danh sách liên tiếp  
    current_list = all_values[i]  
    next_list = all_values[i - 1]  

    # Tìm các giá trị giống nhau  
    common_values = set(current_list) & set(next_list)  

    # In kết quả  
    # # print(f"Các giá trị giống nhau giữa danh sách {i} và danh sách {i + 1}: {common_values}")  
    # print(f"Số lượng giá trị giống nhau giữa danh sách {i} và danh sách {i - 1}: {len(common_values)}")  
    return len(common_values)

def count_same(all_values, i):
    # Chọn danh sách hiện tại (có thể thay đổi chỉ số)  
    current_list = all_values[i]  # Ví dụ: chọn danh sách a  
    # print(f"\nDanh sách hiện tại: {current_list}")  

    # Biến để theo dõi số lượng lớn nhất  
    max_common_count = 0  

    # Kiểm tra số lượng giá trị giống nhau  
    for i, other_list in enumerate(all_values):  
        if other_list is not current_list:  # Chỉ so sánh với danh sách khác  
            # Tìm các giá trị giống nhau  
            common_values = set(current_list) & set(other_list)  
            common_count = len(common_values)  # Đếm số giá trị giống nhau  

            # Cập nhật số lượng lớn nhất nếu cần  
            if common_count > max_common_count:  
                max_common_count = common_count  

    # # In ra số lượng giá trị giống nhau lớn nhất  
    # print(f"Số lượng giá trị giống nhau lớn nhất giữa danh sách hiện tại và các danh sách khác: {max_common_count}")  
    return max_common_count

def format_toml(input_content):
    lines = input_content.splitlines()  # Tách chuỗi thành các dòng

    result = []
    indent_level = 0
    # prev_section = ""
    nhay = []
    flag = True

    for line in lines:
        stripped_line = line.strip()

        if stripped_line.startswith("[") and stripped_line.endswith("]"):
            flag = False
        if flag:
            result.append(stripped_line)
            continue

        if stripped_line.startswith("[") and stripped_line.endswith("]"):
            # flag = False
            nhay.append(split_str(stripped_line))

            if len(nhay) == 1:
                result.append(stripped_line)
                continue


            indent_level = same(nhay, len(nhay)-1)

            
            indent = "  " * indent_level
            result.append(indent + stripped_line)
        else:
            # Câu lệnh hoặc comment giữ nguyên nhưng được thụt lề theo indent_level
            indent = "  " * (indent_level + 1)
            result.append(indent + stripped_line)

    # Trả về kết quả dưới dạng chuỗi
    return "\n".join(result)

# Chuyển đổi nội dung
formatted_content = format_toml(transformed_content)

# In kết quả đã chuyển đổi ra file
output_file = "finish_tom.toml"  # Định nghĩa tên file đầu ra

with open(output_file, "w", encoding="utf-8") as f:
    f.write(formatted_content)

# # In kết quả đã chuyển đổi ra màn hình
# print(formatted_content)

In [7]:
import toml
from deepdiff import DeepDiff

# Đọc file TOML và chuyển thành dictionary
def load_toml(file_path):
    return toml.load(file_path)

# So sánh hai dictionary và in sự khác biệt từng dòng
def compare_dicts(dict1, dict2):
    diff = DeepDiff(dict1, dict2, ignore_order=True)
    
    if not diff:
        print("Hai file TOML giống nhau.")
    else:
        print("Hai file TOML khác nhau.")

        # In ra các giá trị đã thay đổi
        if 'values_changed' in diff:
            print("\nCác giá trị đã thay đổi:")
            for key, change in diff['values_changed'].items():
                print(f"  - {key}:")
                print(f"    - File 1: {change['old_value']}")
                print(f"    - File 2: {change['new_value']}")

        # In ra các mục đã được thêm vào
        if 'dictionary_item_added' in diff:
            print("\nCác mục đã được thêm vào (Add to file 2):")
            for key in diff['dictionary_item_added']:
                print(f"  - {key}")

        # In ra các mục đã bị xóa
        if 'dictionary_item_removed' in diff:
            print("\nCác mục đã bị xóa (Delete in file 2):")
            for key in diff['dictionary_item_removed']:
                print(f"  - {key})")

# Đọc hai file TOML
file1 = '/workspaces/k8sconfig_edit/tom_edit/config.toml'
file2 = 'finish_tom.toml'

data1 = load_toml(file1)
data2 = load_toml(file2)

# So sánh hai dictionary
compare_dicts(data1, data2)


Hai file TOML khác nhau.

Các giá trị đã thay đổi:
  - root['plugins']['io.containerd.grpc.v1.cri']['registry']:
    - File 1: {'config_path': '/etc/containerd/certs.d', 'auths': {}, 'configs': {}, 'headers': {}, 'mirrors': {}}
    - File 2: {'config_path': '/etc/containerd/certs.d'}

Các mục đã bị xóa (Delete in file 2):
  - root['temp'])
  - root['proxy_plugins'])
  - root['plugins']['io.containerd.grpc.v1.cri']['containerd']['default_runtime']['options'])
  - root['plugins']['io.containerd.grpc.v1.cri']['containerd']['untrusted_workload_runtime']['options'])


In [8]:
import toml
import json
from deepdiff import DeepDiff

# Đọc file TOML và chuyển thành dictionary
def load_toml(file_path):
    return toml.load(file_path)

# So sánh hai dictionary và trả về sự khác biệt dưới dạng JSON
def compare_dicts(dict1, dict2):
    diff = DeepDiff(dict1, dict2, ignore_order=True)
    
    result = {}

    if not diff:
        result["message"] = "Same"
    else:
        result["message"] = "Different"
        
        # Nếu có sự thay đổi giá trị
        if 'values_changed' in diff:
            result["values_changed"] = []
            for key, change in diff['values_changed'].items():
                result["values_changed"].append({
                    "key": key,
                    "old_value": change['old_value'],
                    "new_value": change['new_value']
                })

        # Nếu có mục được thêm vào
        if 'dictionary_item_added' in diff:
            result["dictionary_item_added"] = []
            for key in diff['dictionary_item_added']:
                result["dictionary_item_added"].append({
                    "key": key,
                    # "added_in": "file 2"
                })

        # Nếu có mục bị xóa
        if 'dictionary_item_removed' in diff:
            result["dictionary_item_removed"] = []
            for key in diff['dictionary_item_removed']:
                result["dictionary_item_removed"].append({
                    "key": key,
                    # "removed_in": "file 2"
                })

    return result

# Đọc hai file TOML
file1 = '/workspaces/k8sconfig_edit/tom_edit/config.toml'
file2 = 'finish_tom.toml'

data1 = load_toml(file1)
data2 = load_toml(file2)

# So sánh hai dictionary và lấy kết quả
diff_result = compare_dicts(data1, data2)

# Lưu kết quả vào file JSON
with open('diff_result.json', 'w') as json_file:
    json.dump(diff_result, json_file, indent=4)

print("File diff_result.json.")


File diff_result.json.
